#Data Acquisition

Flipkart Website is Web Scarpped using BeautifulSoup and the list of mobile phones is taken from the scarpping ,and the whole data is stored in  a.csv file

In [ ]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import pandas

Taking the url from the Flipkart (Mobile Searching)

In [ ]:
url="https://www.flipkart.com/search?q=mobiles&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&as-pos=2&as-type=RECENT&suggestionId=mobiles%7CMobiles&requestId=8cc7fba1-3802-4aa8-9fd1-a56eba04e23d&as-searchtext=mobiles&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove&p%5B%5D=facets.price_range.from%3D7000&p%5B%5D=facets.price_range.to%3D13000&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&p%5B%5D=facets.ram%255B%255D%3D4%2BGB&p%5B%5D=facets.ram%255B%255D%3D3%2BGB&page="

Finding the Page Response

In [ ]:
page=requests.get(url)
page

<Response [200]>

To Find the Number of Pages

In [ ]:
soup=BeautifulSoup(page.content,'html.parser')#using BeautifulSoup
num_of_pages=int(soup.find('div',class_="_2MImiq").text.replace('12345678910Next','').replace('Page 1 of ',''))
num_of_pages

20

Adding a Loop to get the information of all the pages

In [ ]:
list1=[]
for i in range(1,num_of_pages+1):
  url1=url+str(i)
  page=requests.get(url1)
  soup=BeautifulSoup(page.content,'html.parser')
  lists=soup.find_all('a',class_="_1fQZEK")
  list1.append(lists)


Adding a Loop to get the information from each page 

In [ ]:
info1=[]
for i in range(num_of_pages):
    items=list1[i]
    for item in items:
      record={}
      record['Mobile Name']=item.find('div',class_="_4rR01T").text.split('(')[0]
      record['Color']=item.find('div',class_="_4rR01T").text.split('(')[-1].split(',')[-0]
      record['Storage']=item.find('div',class_="_4rR01T").text.split('(')[-1].split(',')[-1].replace(')','')
      record['Rating']=float(item.find('div',class_="_3LWZlK").text)
      record['Price']=int(item.find('div',class_="_30jeq3 _1_WHN1").text.replace('₹','').replace(',',''))
      info1.append(record)

Finding The Number of Results

In [ ]:
len(info1)

421

Coverting the data into DataFrame

In [ ]:
import pandas as pd
mobiledf=pd.json_normalize(info1)

Displaying The Data Frame(mobiledf)

In [ ]:
mobiledf

,Mobile Name,Color,Storage,Rating,Price
0,POCO C31,Royal Blue,64 GB,4.4,9499
1,REDMI 9i Sport,Metallic Blue,64 GB,4.4,8999
2,SAMSUNG Galaxy F12,Sea Green,64 GB,4.3,11499
3,POCO C31,Shadow Gray,64 GB,4.4,9599
4,realme C25Y,Glacier Blue,64 GB,4.5,10999
...,...,...,...,...,...
416,Redmi Note 5,Black,64 GB,4.4,11999
417,Redmi 6 Pro,Red,32 GB,4.3,7999
418,Infinix Smart 4 Plus,Midnight Black,32 GB,4.4,9999
419,vivo Y81,Gold,32 GB,4.4,13000


Finding the mobile phone which are above Rs 11500

In [ ]:
mobiledf[mobiledf['Price']>11500]

,Mobile Name,Color,Storage,Rating,Price
5,realme C35,Glowing Black,64 GB,4.4,11999
7,realme 9i,Prism Black,64 GB,4.5,12999
12,realme 9i,Prism Blue,64 GB,4.5,12999
13,REDMI Note 10T 5G,Metallic Blue,64 GB,4.4,12999
14,realme C35,Glowing Green,128 GB,4.4,12999
...,...,...,...,...,...
411,Lenovo K8 Plus,Fine Gold,32 GB,4.1,11999
414,Infinix Hot 9,Violet,64 GB,4.4,11999
415,Nokia 3.2,Black,32 GB,4.1,13000
416,Redmi Note 5,Black,64 GB,4.4,11999


Finding the mobile phone which are above Rating 4.3

In [ ]:
mobiledf[mobiledf['Rating']>4.3]

,Mobile Name,Color,Storage,Rating,Price
0,POCO C31,Royal Blue,64 GB,4.4,9499
1,REDMI 9i Sport,Metallic Blue,64 GB,4.4,8999
3,POCO C31,Shadow Gray,64 GB,4.4,9599
4,realme C25Y,Glacier Blue,64 GB,4.5,10999
5,realme C35,Glowing Black,64 GB,4.4,11999
...,...,...,...,...,...
413,SAMSUNG Galaxy J6,Blue,64 GB,4.5,9490
414,Infinix Hot 9,Violet,64 GB,4.4,11999
416,Redmi Note 5,Black,64 GB,4.4,11999
418,Infinix Smart 4 Plus,Midnight Black,32 GB,4.4,9999


Creating a CSV file with the data we scarpped

In [ ]:
pd = pandas.DataFrame(mobiledf)
pd.to_csv("MobileList.csv")